In [ ]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
load_dotenv()
import time
import sql_functions as sf
import psycopg2

In [ ]:
# there are over 80 .csv from airbnb (1 file for 1 city) - this dictionary will help to name the df according to the cities each csv contains 
renaming = {
    0: 'Amsterdam',
    1: 'Antwerp',
    2: 'Asheville',
    3: 'Athens',
    4: 'Austin',
    5: 'Bangkok',
    6: 'Barcelona',
    7: 'Berlin',
    8: 'Bergamo',
    9: 'Belize',
    10: 'Bologna',
    11: 'Boston',
    12: 'Bordeaux',
    13: 'Brussels',
    14: 'Buenos Aires',
    15: 'Cape Town',
    16: 'Cambridge',
    17: 'Chicago',
    18: 'Columbus',
    19: 'Copenhagen',
    20: 'Crete',
    21: 'Dallas',
    22: 'Denver',
    23: 'Dublin',
    24: 'Edinburgh',
    25: 'Florence',
    26: 'Ghent',
    27: 'Geneva',
    28: 'Girona',
    29: 'Hawaii',
    30: 'Hong Kong',
    31: 'Istanbul',
    32: 'Jersey City',
    33: 'Lausanne',
    34: 'Lisbon',
    35: 'London',
    36: 'Los Angeles',
    37: 'Lyon',
    38: 'Madrid',
    39: 'Malaga',
    40: 'Mallorca',
    41: 'Manchester',
    42: 'Melbourne',
    43: 'Menorca',
    44: 'Milan',
    45: 'Montreal',
    46: 'Naples',
    47: 'Nashville',
    48: 'New Orleans',
    49: 'New York City',
    50: 'Newark',
    51: 'Oakland',
    52: 'Oslo',
    53: 'Paris',
    54: 'Portland',
    55: 'Prague',
    56: 'Quebec City',
    57: 'Rhode Island',
    58: 'Riga',
    59: 'Rio de Janeiro',
    60: 'Rome',
    61: 'Rotterdam',
    62: 'Salem',
    63: 'San Diego',
    64: 'San Francisco',
    65: 'Seattle',
    66: 'Sevilla',
    67: 'Santiago',
    68: 'Singapore',
    69: 'Stockholm',
    70: 'Sydney',
    71: 'Taipei',
    72: 'The Hague',
    73: 'Thessaloniki',
    74: 'Tokyo',
    75: 'Toronto',
    76: 'Valencia',
    77: 'Vancouver',
    78: 'Venice',
    79: 'Victoria',
    80: 'Vienna',
    81: 'WashingtonDC',
    82: 'Winnipeg',
    83: 'Zurich',
    84: 'Munich',
    85: 'Mexico City'
}

print(len(renaming))

In [ ]:
# This currency dictionary will help with the currency exchange process
currency_mapping = {
    'Amsterdam': 'EUR',
    'Antwerp': 'EUR',
    'Asheville': 'USD',
    'Athens': 'EUR',
    'Austin': 'USD',
    'Bangkok': 'THB',
    'Barcelona': 'EUR',
    'Berlin': 'EUR',
    'Bergamo': 'EUR',
    'Belize': 'BZD',
    'Bologna': 'EUR',
    'Boston': 'USD',
    'Bordeaux': 'EUR',
    'Brussels': 'EUR',
    'Buenos Aires': 'ARS',
    'Cape Town': 'ZAR',
    'Cambridge': 'USD',
    'Chicago': 'USD',
    'Columbus': 'USD',
    'Copenhagen': 'DKK',
    'Crete': 'EUR',
    'Dallas': 'USD',
    'Denver': 'USD',
    'Dublin': 'EUR',
    'Edinburgh': 'GBP',
    'Florence': 'EUR',
    'Ghent': 'EUR',
    'Geneva': 'CHF',
    'Girona': 'EUR',
    'Hawaii': 'USD',
    'Hong Kong': 'HKD',
    'Istanbul': 'TRY',
    'Jersey City': 'USD',
    'Lausanne': 'CHF',
    'Lisbon': 'EUR',
    'London': 'GBP',
    'Los Angeles': 'USD',
    'Lyon': 'EUR',
    'Madrid': 'EUR',
    'Malaga': 'EUR',
    'Mallorca': 'EUR',
    'Manchester': 'GBP',
    'Melbourne': 'AUD',
    'Menorca': 'EUR',
    'Milan': 'EUR',
    'Montreal': 'CAD',
    'Naples': 'EUR',
    'Nashville': 'USD',
    'New Orleans': 'USD',
    'New York City': 'USD',
    'Newark': 'USD',
    'Oakland': 'USD',
    'Oslo': 'NOK',
    'Paris': 'EUR',
    'Portland': 'USD',
    'Prague': 'CZK',
    'Quebec City': 'CAD',
    'Rhode Island': 'USD',
    'Riga': 'EUR',
    'Rio de Janeiro': 'BRL',
    'Rome': 'EUR',
    'Rotterdam': 'EUR',
    'Salem': 'USD',
    'San Diego': 'USD',
    'San Francisco': 'USD',
    'Seattle': 'USD',
    'Sevilla': 'EUR',
    'Singapore': 'SGD',
    'Stockholm': 'SEK',
    'Sydney': 'AUD',
    'Taipei': 'TWD',
    'The Hague': 'EUR',
    'Thessaloniki': 'EUR',
    'Tokyo': 'JPY',
    'Toronto': 'CAD',
    'Valencia': 'EUR',
    'Vancouver': 'CAD',
    'Venice': 'EUR',
    'Victoria': 'CAD',
    'Vienna': 'EUR',
    'WashingtonDC': 'USD',
    'Winnipeg': 'CAD',
    'Zurich': 'CHF',
    'Munich': 'EUR',
    'Mexico City': 'MXN',
    'Santiago': 'CLP'
}
print(len(currency_mapping))

In [ ]:
# function for the currency converter API - it is set to convert to EUR
def get_euro_values(currency_old, amount):
	url = "https://currency-converter-by-api-ninjas.p.rapidapi.com/v1/convertcurrency"
	querystring = {"have": currency_old,"want":"EUR","amount":amount}
	api_key = os.getenv('currency_api_key') # extract the value for the api key
	headers = {
		"X-RapidAPI-Key": api_key,
		"X-RapidAPI-Host": "currency-converter-by-api-ninjas.p.rapidapi.com"}
	try:
		response = requests.get(url, headers=headers, params=querystring)
		response.raise_for_status()  # Raise an exception for HTTP errors
		response_json = response.json()
		response_json = pd.json_normalize(response_json)
		return response_json['new_amount'][0]
		print(response_json)
# the upcoming parts help with the error handling if the API isnt working as it should
	except requests.HTTPError as e:
		print("HTTP Error occurred:", e)
	except json.JSONDecodeError as e:
		print("Error decoding JSON:", e)
	except requests.RequestException as e:
		print("Error connecting to the API:", e)

In [ ]:
# Call the currency exchange API to get the exchange rates for the unique currencies in currency_mapping dict
conversion_rate_dict = {}
for currency in set(currency_mapping.values()):
    euro_value = get_euro_values(currency, 100)
    converted_value = round(euro_value / 100, 4)
    conversion_rate_dict[currency] = converted_value

In [ ]:
print(conversion_rate_dict)

In [ ]:
# saving the content under another variable
conversion_rate = conversion_rate_dict
print(type(conversion_rate))

In [ ]:
# Create data frames for all 80+ cities + adding and dropping of columns
keep_columns = ['latitude', 'longitude', 'room_type', 'price']
for num in range(len(renaming)):
    city_name = renaming[num]
    df_name = f"df_{city_name.replace(' ', '_').lower()}"
    csv_file = f"data/airbnb/listings ({num}).csv"
    globals()[df_name] = pd.read_csv(csv_file, sep=',', header=0)
    globals()[df_name] = globals()[df_name][keep_columns]  # Keep only specified columns
    globals()[df_name]['city'] = city_name  # Add 'city' column with city name
    globals()[df_name]['currency_old'] = currency_mapping[city_name]  # Add 'currency' column with currency value
    globals()[df_name]['converted_price'] = conversion_rate[globals()[df_name]['currency_old'][0]] * globals()[df_name]['price']  # call function for the currency converter
    #globals()[df_name]['currency_new'] = get_euro_values(currency_mapping[city_name], globals()[df_name]['price'])  # call function for the currency converter
    #globals()[df_name]['currency_new'] = get_euro_values('E', 500)  # call function for the currency converter
    #time.sleep(1)

# Print all data frames with the new columns
for num in range(len(renaming)):
    city_name = renaming[num]
    df_name = f"df_{city_name.replace(' ', '_').lower()}" #replace the white spaces and apply lowercase to all names
    print(eval(df_name))
    print()


In [ ]:
# some random dfs to check the current progress
df_columbus
df_zurich
df_munich
df_winnipeg
df_tokyo

In [ ]:
# using the globals built-in-function "globals" to dynamicly access the 80+ dfs for each city
globals()[df_name]

In [ ]:
globals()[df_name].describe()

In [ ]:
# check if converted price still has more than one value
print(type(df_amsterdam["converted_price"][0]))

In [ ]:
print(type(df_amsterdam))

In [ ]:
df_amsterdam

In [ ]:
# Print the AVG price for each room type and each city
for num in range(len(renaming)):
    city_name = renaming[num]
    df_name = f"df_{city_name.replace(' ', '_').lower()}"
    #globals()[df_name] = eval(df_name).groupby('room_type').converted_price.mean().to_frame()
    globals()[df_name] = pd.pivot_table(eval(df_name).groupby('room_type').converted_price.mean().to_frame(), values='converted_price', columns='room_type')
    #globals()[df_name] = pd.pivot_table(eval(df_name), values='converted_price', index='room_type', aggfunc='mean')
    globals()[df_name]['city'] = city_name  # add 'city' column with city name
    print(globals()[df_name])

In [ ]:
# create a new df variable for the concatenated dfs
all_cities_dfs = []

# Concatenate all the df
for num in range(len(renaming)):
    city_name = renaming[num]
    df_name = f"df_{city_name.replace(' ', '_').lower()}"
    all_cities_dfs.append(globals()[df_name])

# Concatenate the DataFrames in the list
combined_df = pd.concat(all_cities_dfs)

# Display the entire DataFrame without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

combined_df

In [ ]:
type(combined_df )

In [ ]:
# # upload the airbnb prices for all cities to the sql database as "airbnb_prices_all"
# schema = 'capstone_travel_index'
# engine = sf.get_engine()
# table_name = 'airbnb_prices_all'
# if engine!=None:
#     try:
#         combined_df.to_sql(name=table_name, # Name of SQL table
#                         con=engine, # Engine or connection
#                         if_exists='replace', # Drop the table before inserting new values 
#                         schema=schema, # Use schmea that was defined earlier
#                         index=False, # Write DataFrame index as a column
#                         chunksize=5000, # Specify the number of rows in each batch to be written at a time
#                         method='multi') # Pass multiple values in a single INSERT clause
#         print(f"The {table_name} table was imported successfully.")
#     # Error handling
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#         engine = None

In [1]:
# #save the dataset "df_items" as .csv to the local folder so it can be uploaded to github
# combined_df.to_csv('data/airbnb_prices_all')

NameError: name 'combined_df' is not defined